In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [3]:
# Veri dönüşümlerini tanımlayın (boyutlandırma, normalizasyon)
transform = transforms.Compose([
    transforms.Resize((128, 128)),# Görüntü boyutunu ayarlayın
    transforms.ToTensor(),# Görüntüyü Tensor formatına dönüştürün
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize edin
])

In [4]:
def is_valid_image(file_path):
    try:
        with open(file_path, 'rb') as f:
            img = Image.open(f)
            img.load()
        return True
    except:
        return False

root_folder = "PetImages_Train"  # Verilerin bulunduğu klasörü belirtin
dataset = datasets.ImageFolder(root=root_folder, transform=transform, is_valid_file=is_valid_image)

dataloader = DataLoader(dataset=dataset , batch_size=32 ,shuffle=True)

In [5]:
dataset.class_to_idx

{'Cat': 0, 'Dog': 1}

In [6]:
for input ,labels in dataloader:
    print(input)
    print(labels)
    break

tensor([[[[-0.4824, -0.5137, -0.5922,  ..., -0.7255, -0.6941, -0.7412],
          [-0.6549, -0.5529, -0.5451,  ..., -0.6863, -0.6627, -0.6314],
          [-0.6863, -0.5765, -0.5922,  ..., -0.6000, -0.4980, -0.5216],
          ...,
          [-0.4196, -0.4196, -0.5137,  ..., -0.3333, -0.3020, -0.3020],
          [-0.4196, -0.5059, -0.6157,  ..., -0.3412, -0.3255, -0.3098],
          [-0.4824, -0.4824, -0.4118,  ..., -0.3490, -0.3490, -0.3412]],

         [[-0.4510, -0.4902, -0.5686,  ..., -0.5843, -0.6078, -0.6941],
          [-0.6392, -0.5373, -0.5216,  ..., -0.6078, -0.6314, -0.6157],
          [-0.6784, -0.5608, -0.5765,  ..., -0.5843, -0.5059, -0.5137],
          ...,
          [-0.2078, -0.2078, -0.3255,  ..., -0.1922, -0.1608, -0.1686],
          [-0.2000, -0.2941, -0.4353,  ..., -0.2000, -0.1373, -0.1686],
          [-0.2549, -0.2627, -0.2235,  ..., -0.2078, -0.1608, -0.1922]],

         [[-0.8353, -0.8431, -0.8980,  ..., -0.9529, -0.9373, -0.9529],
          [-0.9373, -0.8510, -

In [7]:
# CNN modelini oluşturun
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # 3 kanal giriş, 16 çıkış, 3x3 çekirdek
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 2x2 maksimum havuzlama
            nn.Conv2d(16, 32, 3, padding=1),  # 16 kanal giriş, 32 çıkış, 3x3 çekirdek
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),  # 32 kanal giriş, 64 çıkış, 3x3 çekirdek
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),  # Düzleştirme işlemi
            nn.Linear(64 * 16 * 16, 512),  # Tam bağlantılı katman
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout katmanı (aşırı uyumu önlemek için)
            nn.Linear(512, 2)  # İki sınıflı çıkış katmanı (kedi ve köpek)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Modeli oluşturun
model = CNNModel().to("cuda")

# Kayıp fonksiyonunu ve optimizasyon yöntemini tanımlayın
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Eğitim döngüsü
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in dataloader:
        inputs ,labels = inputs.to("cuda"), labels.to("cuda")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    print(f"Loss : {loss}")
    print(f"Loss_data : {loss.data}")
    print(f"Loss_item : {loss.item()}")

print("Eğitim tamamlandı.")

Loss : 0.6595098376274109
Loss_data : 0.6595098376274109
Loss_item : 0.6595098376274109
Loss : 0.5590130686759949
Loss_data : 0.5590130686759949
Loss_item : 0.5590130686759949
Loss : 0.5436583161354065
Loss_data : 0.5436583161354065
Loss_item : 0.5436583161354065
Loss : 0.5218498706817627
Loss_data : 0.5218498706817627
Loss_item : 0.5218498706817627
Loss : 0.48614782094955444
Loss_data : 0.48614782094955444
Loss_item : 0.48614782094955444
Eğitim tamamlandı.


In [9]:
cat_path = "C:/Users/emrer/Desktop/python-uygulamalar/catvsdog/catphoto_1.jpeg"

In [10]:
cat_img = Image.open(cat_path)
type(cat_img)

PIL.JpegImagePlugin.JpegImageFile

In [11]:
transform_animal = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),    
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [12]:
cat_transform = transform_animal(cat_img)

In [13]:
cat_transform = cat_transform.to("cuda")

In [14]:
cat_transform = cat_transform.view(-1,3,128,128)

In [15]:
cat_transform.shape

torch.Size([1, 3, 128, 128])

In [16]:
model.eval()
with torch.no_grad():
    print(model(cat_transform))

tensor([[ 0.4717, -0.3605]], device='cuda:0')


In [17]:
_,predicted = torch.max(model(cat_transform),1)

In [18]:
predicted

tensor([0], device='cuda:0')

In [19]:
dog_path = "C:/Users/emrer/Desktop/python-uygulamalar/catvsdog/dogphoto_2.jpg"

In [20]:
dog_img = Image.open(dog_path)
type(dog_img)

PIL.JpegImagePlugin.JpegImageFile

In [21]:
dog_transform = transform_animal(dog_img)

In [22]:
dog_transform = dog_transform.to("cuda")

In [23]:
dog_transform = dog_transform.view(-1,3,128,128)

In [24]:
model.eval()
with torch.no_grad():
    print(model(dog_transform))

tensor([[-1.1594,  1.1541]], device='cuda:0')


In [25]:
_,predicted_2 = torch.max(model(dog_transform),1)

In [26]:
predicted_2

tensor([1], device='cuda:0')